#  Задание 4

Для нормального распределения $X_i \sim N(\theta, \sigma ^2)$, если взять сумму независимых $X_i$, то доверительный интервал для $\theta$ будет:
$$P_{\theta}\left(\overline{x} - \frac{c_{1-\frac{\alpha}{2}} \sigma}{\sqrt{n}} < \theta < \overline{x} + \frac{c_{1-\frac{\alpha}{2}} \sigma}{\sqrt{n}} \right) = 1 - \alpha$$

Для **параметрического бутстрэпа** - оценим $\hat{\theta}$ и $\hat{\sigma ^2}$ по выборке и построим дополнительные выборки из распределения $X_i \sim N(\hat{\theta}, \hat{\sigma ^2})$

Для **не параметрического бутстрэпа** - из исходной выборки будем делать случайный выбор с возвратом.

## сделаем пункты 1-4 на 1 выборке

In [1]:
import numpy as np
from scipy import stats # чтобы считать квантили

In [2]:
def ci_theoretical(data, alpha=0.05):
    """дает кортеж из нижней и верхней границы доверительного интервала для среднего в нормальных x_i

    Args:
        data (array like): данные sample
        alpha (float, optional): уровень значимости. Defaults to 0.05.
    """
    std = np.std(data, ddof=1, dtype=np.float64)
    n = len(data)
    avg = np.mean(data)


    CI_Theoretical = (avg + stats.norm.ppf(alpha/2)*std /np.sqrt(n), avg + stats.norm.ppf(1-alpha/2)*std /np.sqrt(n))
    return CI_Theoretical


In [3]:

n = 50 # размер выборки 
alpha = 0.05 # параметр ошибки

np.random.seed(80085)
theta = np.random.uniform(10,20) # параметр тета
theor_sigma = np.random.uniform(1,2) # параметр тета
print((theta, theor_sigma))

(13.551121285103044, 1.0281182470223413)


In [4]:
sample1 = np.random.normal(theta,theor_sigma,size=n) 
sample1

array([13.101581  , 12.9629306 , 14.15844439, 13.1420225 , 13.21470771,
       12.92822876, 13.73439605, 13.62612039, 13.68009226, 12.11077405,
       13.64413202, 13.68589658, 11.13375744, 13.96078479, 11.5580977 ,
       14.58205901, 12.50967174, 14.29788902, 13.26076637, 13.87741364,
       13.49368701, 14.84815911, 15.17284706, 14.50394078, 12.42229787,
       14.07777694, 13.74842455, 14.05886051, 12.5529233 , 13.25924115,
       12.84479367, 13.04266122, 12.92148125, 12.97844854, 12.87426001,
       13.14309871, 12.68231808, 11.87790938, 13.13715342, 13.93000191,
       15.00731573, 11.89047398, 15.30802172, 12.37235166, 13.16092178,
       13.49004403, 14.48894427, 13.98446822, 14.10699862, 14.0301892 ])

3.1) теоретический доверительный интервал

In [5]:
ci_result_theor = ci_theoretical(sample1)
ci_result_theor

(13.158376544840669, 13.6648146417907)

3.2) параметрический бутстрэп доверительный интервал

In [6]:
def ci_param_bootstrap(data, alpha=0.05, number_of_bootstrap_samples=10, size_of_bootstrap_samples=20 ):
    """параметрический бутстрэп

    Args:
        data (array like): данные для оценки среднего
        alpha (float, optional): увроень доверия. Defaults to 0.05.
        number_of_bootstrap_samples (int, optional): сколько сэмплов для бутстрэпа делать. Defaults to 10.
        size_of_bootstrap_samples (int, optional): сколько наблюдений делать в каждый сэмпл. Defaults to 20.
    """
    # Оцениваем неизвестный параметр theta 
    sample_mean = np.mean(data) 
    sample_std = np.std(data, ddof=1)
    # print(sample_mean, sample_std)

    # Генерируем выборку из распределения N(sample_mean, sigma)
    bootstrap_samples = np.random.normal(sample_mean,sample_std,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) 

    #  Считаем среднее для каждой выборки 
    bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples)

    # Вычисляем параметрический бутстрэп доверительный интервал
    CI_Bootstrap_Parametric = (np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2))

    return(CI_Bootstrap_Parametric)
    
ci_result_parambs = ci_param_bootstrap(sample1)
ci_result_parambs

(13.191500451623982, 13.844226852338117)

3.3)  непараметрический бутстрэп интервал

In [7]:

def ci_non_param_bootstrap(data, alpha=0.05, number_of_bootstrap_samples=10, size_of_bootstrap_samples=20 ):
    """непараметрический бутстрэп

    Args:
        data (array like): данные для оценки среднего
        alpha (float, optional): увроень доверия. Defaults to 0.05.
        number_of_bootstrap_samples (int, optional): сколько сэмплов для бутстрэпа делать. Defaults to 10.
        size_of_bootstrap_samples (int, optional): сколько наблюдений делать в каждый сэмпл. Defaults to 20.
    """

    # Генерируем выборку из исходного распределения
    bootstrap_samples = np.random.choice(data,size=[number_of_bootstrap_samples,size_of_bootstrap_samples]) 

    #  Считаем среднее для каждой выборки 
    bootstrap_estimates = np.apply_along_axis(np.mean, 1, bootstrap_samples)

    # Вычисляем параметрический бутстрэп доверительный интервал
    CI_Bootstrap_Parametric = (np.quantile(bootstrap_estimates,alpha/2), np.quantile(bootstrap_estimates,1-alpha/2))

    return(CI_Bootstrap_Parametric)

ci_result_non_parambs = ci_non_param_bootstrap(sample1)
ci_result_non_parambs

(13.173227723425756, 13.820911172277498)

## сравним их длины

In [8]:
print(f'Длина интервала для теоретического доверительного интервала {ci_result_theor[1] - ci_result_theor[0]:.4f}')
print(f'Длина интервала для параметрического бутстрэпа {ci_result_parambs[1] - ci_result_parambs[0]:.4f}')
print(f'Длина интервала для непараметрического бутстрэпа {ci_result_non_parambs[1] - ci_result_non_parambs[0]:.4f}')

Длина интервала для теоретического доверительного интервала 0.5064
Длина интервала для параметрического бутстрэпа 0.6527
Длина интервала для непараметрического бутстрэпа 0.6477


Для этого случайного зерна и параметров, мы получили то, что длина теоретического доверительного интервала самай коротка, потом идет непараметрический бутстрэп, потом параметрический бутстрэп (самый широкий интервал)

## Сделаем это 10000 раз

In [9]:
import pandas as pd 
from tqdm.notebook import trange

In [10]:
iterations = 10000

theoretical = np.zeros([iterations,2]) # здесь будем хранить результаты для теоретического доверительного интервала
parametric_bootstrap = np.zeros([iterations,2]) # здесь будем хранить результаты для параметрического бутстрэпа 
nonparametric_bootstrap = np.zeros([iterations,2]) # здесь будем хранить результаты для непараметрического бутстрэпа 

for i in trange(iterations, desc='calc CI'):
    sample4 = np.random.normal(theta,theor_sigma,size=n) 

    CI_Theoretical = ci_theoretical(sample4,alpha)
    theoretical[i,0] = (theta >= CI_Theoretical[0]) and (theta <= CI_Theoretical[1])
    theoretical[i,1] = CI_Theoretical[1] - CI_Theoretical[0]

    CI_param = ci_param_bootstrap(sample4,alpha)
    parametric_bootstrap[i,0] = (theta >= CI_param[0]) and (theta <= CI_param[1])
    parametric_bootstrap[i,1] = CI_param[1] - CI_param[0]

    CI_nonparam = ci_non_param_bootstrap(sample4,alpha)
    nonparametric_bootstrap[i,0] = (theta >= CI_nonparam[0]) and (theta <= CI_nonparam[1])
    nonparametric_bootstrap[i,1] = CI_nonparam[1] - CI_nonparam[0]





calc CI:   0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
prop_correct = {}
average_length = {}

prop_correct['theoretical'], average_length['theoretical'] = np.mean(theoretical, axis=0)
prop_correct['param'], average_length['param'] = np.mean(parametric_bootstrap, axis=0)
prop_correct['non_param'], average_length['non_param'] = np.mean(nonparametric_bootstrap, axis=0)


df = pd.DataFrame.from_records([prop_correct, average_length], index = ['доля попадений в CI', 'длина CI средняя']).T
df

,доля попадений в CI,длина CI средняя
theoretical,0.9455,0.566495
param,0.9159,0.645839
non_param,0.9151,0.638082


Ситуация не поменялась содержательно: параметрический самый широкий интервал, теоретический самый узкий. При этом вероятность попадания в доверительный интервал, ближе всего к $1-\alpha$ для теоретического интервала, и дальше всего от ожидаемого значения в 0,95 для непараметрического интервала.